In [6]:
import os
from ultralytics import YOLO
import cv2

# Set up the video capture for the webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Path to the trained model
model_path = os.path.join('.', 'weights', 'last.pt')

# Load the custom model
model = YOLO(model_path)

threshold = 0.5

# Get the screen dimensions
screen_width = 1280  # Replace with your screen width
screen_height = 720  # Replace with your screen height

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    results = model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            label = results.names[int(class_id)].upper()
            color = (0, 255, 0) if label != "WRONG" else (0, 0, 255)  # Green for correct, red for wrong

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

    # Resize the frame to fit within the screen dimensions
    frame_height, frame_width = frame.shape[:2]
    aspect_ratio = frame_width / frame_height

    if frame_width > screen_width or frame_height > screen_height:
        if aspect_ratio > 1:
            new_width = screen_width
            new_height = int(new_width / aspect_ratio)
        else:
            new_height = screen_height
            new_width = int(new_height * aspect_ratio)

        frame = cv2.resize(frame, (new_width, new_height))
    else:
        frame = cv2.resize(frame, (screen_width, screen_height))

    # Display the frame with detection boxes
    cv2.imshow('YOLO Real-Time Detection', frame)

    # Press 'q' to exit the real-time detection
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



0: 480x640 1 Wrong, 185.0ms
Speed: 3.0ms preprocess, 185.0ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wrong, 155.0ms
Speed: 0.0ms preprocess, 155.0ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wrong, 133.0ms
Speed: 1.4ms preprocess, 133.0ms inference, 10.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wrong, 183.2ms
Speed: 0.0ms preprocess, 183.2ms inference, 7.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wrong, 169.0ms
Speed: 3.4ms preprocess, 169.0ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wrong, 167.7ms
Speed: 3.5ms preprocess, 167.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wrong, 172.3ms
Speed: 0.0ms preprocess, 172.3ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Wrong, 166.0ms
Speed: 2.7ms preprocess, 166.0ms inference, 3.6ms postprocess per image at shape 

In [8]:
import os
from ultralytics import YOLO
import cv2
import serial
import serial.tools.list_ports
import time

arduino_port = 'COM3'
baud_rate = 9600
ser = serial.Serial(arduino_port, baud_rate)
time.sleep(2)  # Wait for the serial connection to initialize

# Set up the video capture for the webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Path to the trained model
model_path = os.path.join('.', 'weights', 'last.pt')

# Load the custom model
model = YOLO(model_path)

threshold = 0.5

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    results = model(frame)[0]
    wrong_detected = False

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            label = results.names[int(class_id)].upper()
            color = (0, 255, 0) if label != "WRONG" else (0, 0, 255)
            if label == "WRONG":
                wrong_detected = True

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 4)
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)

    if wrong_detected:
        ser.write(b'W')  # Send signal to Arduino

    # Display the frame with detection boxes
    cv2.imshow('YOLO Real-Time Detection', frame)

    # Press 'q' to exit the real-time detection
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
ser.close()
cv2.destroyAllWindows()



0: 480x640 (no detections), 204.0ms
Speed: 185.8ms preprocess, 204.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 202.5ms
Speed: 2.0ms preprocess, 202.5ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
